In [3]:
#生成/转换/仿制
import torch
import numpy as np

data = [[1, 2],[3, 4]]
np_data = np.array(data)
t_data  = torch.tensor(data)

tensor = torch.rand(3, 4)
print(f"tensor.shape:{tensor.shape}, tensor.dtype:{tensor.dtype}, tensor.device:{tensor.device}")

np2t = torch.from_numpy(np_data)
x_ones = torch.ones_like(np2t)                    # retains the properties of x_data
x_rand = torch.rand_like(np2t, dtype=torch.float) # overrides the datatype of x_data

tensor.shape:torch.Size([3, 4]), tensor.dtype:torch.float32, tensor.device:cpu


In [17]:
#行列操作
tensor = torch.ones(2, 10)
tensor[:,1] = 0
print(tensor)

tensor([[1., 0., 1., 1., 1., 1., 1., 1., 1., 1.],
        [1., 0., 1., 1., 1., 1., 1., 1., 1., 1.]])


In [5]:
import torch; torch.arange(13).chunk(6)

(tensor([0, 1, 2]),
 tensor([3, 4, 5]),
 tensor([6, 7, 8]),
 tensor([ 9, 10, 11]),
 tensor([12]))

In [22]:
#运算
x = torch.ones((2, 2)) * 2
y = torch.ones((2, 2)) * 3
print(f"x.mul(y)      :{x.mul(y).flatten()}")      # This computes the element-wise product
print(f"x * y         :{(x * y).flatten()}")       # Alternative syntax
print(f"x.matmul(y.T) :{x.matmul(y.T).flatten()}") # Alternative syntax:
print(f"x.mm(y)       :{(x.mm(y.T).flatten())}")
print(f"x @ y.T       :{(x @ y.T).flatten()}")


x.mul(y)      :tensor([6., 6., 6., 6.])
x * y         :tensor([6., 6., 6., 6.])
x.matmul(y.T) :tensor([12., 12., 12., 12.])
x.mm(y)       :tensor([12., 12., 12., 12.])
x @ y.T       :tensor([12., 12., 12., 12.])


In [16]:
tensor = torch.ones(4, 4)
print(tensor.flatten())
tensor.mul_(5)
print(tensor.flatten())

tensor([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.])
tensor([5., 5., 5., 5., 5., 5., 5., 5., 5., 5., 5., 5., 5., 5., 5., 5.])


In [19]:
# [Attention]: Tensors on the CPU and NumPy arrays can share their underlying memory locations, 
# and changing one will change the other.
t = torch.ones(10)
print(f"t: {t}")
n = t.numpy()
n[0] = 0
print(f"t: {t}")

t: tensor([1., 1., 1., 1., 1., 1., 1., 1., 1., 1.])
t: tensor([0., 1., 1., 1., 1., 1., 1., 1., 1., 1.])


In [26]:
#random init tensor
from timm.models.layers import trunc_normal_
x = torch.zeros((1, 3, 255, 255))
trunc_normal_(x, mean=.02, std=.02)
print(torch.mean(x), torch.std(x))

tensor(0.0200) tensor(0.0200)


In [ ]:
import torch
x = torch.zeros(4,4,4,4,4)
i = torch.ones(100, dtype=torch.long)
j = torch.ones(100, dtype=torch.long)
print(x[i][j].shape)

In [ ]:
x = torch.arange(0, 4).reshape(2, 2)
i = torch.zeros(2000, dtype=torch.long)
print(x)
print(x[0])
print(x[i])
print(x[i].shape)
#[169, 3] --> [2401, 3] --> [49, 49, 3]

In [7]:
import torch
w_size = 7; num_heads = 12
# torch.zeros((2 * w_size - 1) * (2 * w_size - 1, num_heads))

RuntimeError: Storage size calculation overflowed with sizes=[13, 12, 13, 12, 13, 12, 13, 12, 13, 12, 13, 12, 13, 12, 13, 12, 13, 12, 13, 12, 13, 12, 13, 12, 13, 12]